# Fake News Detection - N-gram Vectorization (Content)

## Data Preparation

In [1]:
import time

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

import re
import nltk
import json
from nltk.stem import WordNetLemmatizer

### Import News Content as Input and News Label as Output

In [2]:
start_time = time.time()

npz_content = np.load('Content_Data.npz',allow_pickle=True)

content = npz_content['inputs']
output = npz_content['targets']

content.shape, output.shape

((9805,), (9805,))

### Text Preprocessing
1. Remove Special Characters using Regular Expressions
2. Tokenize text
3. Word Normalization (Lemmatization)

In [3]:
lemmatizer = WordNetLemmatizer()

cleaned_content = []

for j in range(content.shape[0]):
    content_sentence = re.sub('[^A-Za-z0-9 ]+', '', content[j])
    content_tokens = nltk.word_tokenize(content_sentence)
    content_lemmatized_word = [lemmatizer.lemmatize(word) for word in content_tokens]
    cleaned_content.append(" ".join(word.lower() for word in content_lemmatized_word))

cleaned_content = np.asarray(cleaned_content, dtype=object)

### Split Train and Test Data

In [4]:
X_train_content, X_test_content, y_train_content, y_test_content = train_test_split(cleaned_content, output, test_size = 0.2, random_state = 1)

### Learn Vocabulary from Training Text and Vectorize the Training Texts

In [5]:
max_vocab_content = 30000

# Create TF-IDF of news content
tfidf_content = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS,ngram_range=(1,3), max_features=max_vocab_content)

sparse_tfidf_train_content = tfidf_content.fit_transform(X_train_content)

X_tfidf_train_content = sparse_tfidf_train_content.toarray()

### Create and Train a Simple Neural Network Model with News Content

In [6]:
model_content = tf.keras.Sequential()
model_content.add(tf.keras.layers.Dense(64,input_shape=(max_vocab_content,),activation='relu'))
model_content.add(tf.keras.layers.Dropout(0.2))
model_content.add(tf.keras.layers.Dense(16,activation='relu'))
model_content.add(tf.keras.layers.Dropout(0.2))
model_content.add(tf.keras.layers.Dense(1,activation='relu'))
model_content.add(tf.keras.layers.Activation('sigmoid'))

model_content.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy', 'Recall'])

batch_size = 200
max_epochs = 10
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model_content.fit(X_tfidf_train_content, y_train_content, batch_size = batch_size, epochs = max_epochs, callbacks=[early_stopping], validation_split = 0.2 , shuffle=True, verbose=1)

Train on 6275 samples, validate on 1569 samples
Epoch 1/10
6275/6275 [==============================] - 2s 320us/sample - loss: 0.6698 - accuracy: 0.7139 - Recall: 0.9180 - val_loss: 0.6251 - val_accuracy: 0.7960 - val_Recall: 0.9937
Epoch 2/10
6275/6275 [==============================] - 1s 188us/sample - loss: 0.5749 - accuracy: 0.8379 - Recall: 0.9915 - val_loss: 0.5354 - val_accuracy: 0.8572 - val_Recall: 0.9874
Epoch 3/10
6275/6275 [==============================] - 1s 185us/sample - loss: 0.4831 - accuracy: 0.9084 - Recall: 0.9902 - val_loss: 0.4852 - val_accuracy: 0.8917 - val_Recall: 0.9785
Epoch 4/10
6275/6275 [==============================] - 1s 188us/sample - loss: 0.4239 - accuracy: 0.9554 - Recall: 0.9945 - val_loss: 0.4781 - val_accuracy: 0.8808 - val_Recall: 0.9798
Epoch 5/10
6275/6275 [==============================] - 1s 187us/sample - loss: 0.3928 - accuracy: 0.9750 - Recall: 0.9967 - val_loss: 0.4832 - val_accuracy: 0.8872 - val_Recall: 0.9810
Epoch 6/10
6275/6275 [

### Convert Test Data into Similar Format to Feed into Neural Network Model Trained

In [7]:
sparse_tfidf_test_content = tfidf_content.transform(X_test_content)

X_tfidf_test_content = sparse_tfidf_test_content.toarray()

### Test Performance on Neural Network Model (Accuracy)

In [8]:
model_content_loss, model_content_accuracy, model_content_recall = model_content.evaluate(X_tfidf_test_content, y_test_content)

1961/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [9]:
model_content_loss, model_content_accuracy, model_content_recall

(0.4763474156818847, 0.9031107, 0.9748491)

In [10]:
y_pred_content = model_content.predict_classes(X_tfidf_test_content)

con_mat_content = tf.math.confusion_matrix(labels=y_test_content, predictions=y_pred_content).numpy()

In [11]:
con_mat_content

array([[802, 165],
       [ 25, 969]])

In [12]:
end_time = time.time()

time_elapsed = end_time - start_time
time_elapsed

71.28125262260437

In [13]:
tfidf_model_performance = {
                                "Content_Accuracy": str(model_content_accuracy),
                                "Content_Recall": str(model_content_recall),
                                "Time": str(time_elapsed)
                          }

with open('Model_TFIDF_Content.json', 'w') as Model_TFIDF_file:
    json.dump(tfidf_model_performance, Model_TFIDF_file)